<a href="https://colab.research.google.com/github/pieper/Notebooks/blob/master/TCIA_to_NII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gcloud auth login



Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?code_challenge=mYFYT-mfcigjN7QTI9tqNSkoMrWvXU1d5J5FJJ_qm0k&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


Enter verification code: 4/vgEAhDZKmFtm98jA6fwmFlecJPA4TaMsZ4aQAkBpmSslAhb7Nsxmuhk

You are now logged in as [pieper@isomics.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


Convert each series into a nifti file

In [ ]:
# TODO: make a small lib in a github repo


import subprocess

def run(cmd):
  child = subprocess.run(cmd.split(), 
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE,
                            check=True)
  return child.stdout.decode().strip()

def freshToken():
  token = run('gcloud auth print-access-token')
  return token

def healthcareDatasets():
  return run('gcloud beta healthcare datasets list --project chc-tcia').split()[3::3]

try:
  run("gsutil mb -p idc-sandbox-000 gs://dev-isomics-scripts-tcia-nii")
  print('bucket created')
except subprocess.CalledProcessError:
  print('bucket already exists')

collections = healthcareDatasets()
print(collections)


bucket already exists
['4d-lung', 'anti-pd-1_melanoma', 'apollo', 'breast-diagnosis', 'breast-mri-nact-pilot', 'cbis-ddsm', 'cc-radiomics-phantom', 'cptac-ccrcc', 'cptac-cm', 'cptac-gbm', 'cptac-hnscc', 'cptac-lscc', 'cptac-luad', 'cptac-pda', 'cptac-ucec', 'ct-colonography', 'ct-lymph-nodes', 'head-neck-cetuximab', 'head-neck-pet-ct', 'hnscc', 'hnscc-3dct-rt', 'ispy1', 'ivygap', 'lctsc', 'lgg-1p19qdeletion', 'lidc-idri', 'lung-fused-ct-pathology', 'lung-phantom', 'lungct-diagnosis', 'mouse-astrocytoma', 'mouse-mammary', 'mri-dir', 'naf-prostate', 'nsclc-radiogenomics', 'nsclc-radiomics', 'nsclc-radiomics-genomics', 'pancreas-ct', 'phantom-fda', 'prostate-3t', 'prostate-diagnosis', 'prostate-fused-mri-pathology', 'prostatex', 'qiba-ct-1c', 'qin-breast-dce-mri', 'qin-headneck', 'qin-lung-ct', 'qin-pet-phantom', 'rembrandt', 'rider-breast-mri', 'rider-lung-ct', 'rider-lung-pet-ct', 'rider-neuro-mri', 'rider-phantom-mri', 'rider-phantom-pet-ct', 'soft-tissue-sarcoma', 'spie-aapm-lung-ct-c

In [ ]:

# create a python script for processing
# TODO: this should pulled from a github repo
pythonScriptTemplate = """


import dicomweb_client
import os
import pydicom
import random
import subprocess
import sys

def run(cmd):
  child = subprocess.run(cmd.split(), 
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE,
                            check=True)
  return child.stdout.decode().strip()

def freshToken():
    token = run('gcloud auth print-access-token')
    return token

def freshClient(url):
    token = freshToken()
    headers = {
        "Authorization" : "Bearer %s" % freshToken()
    }
    return dicomweb_client.api.DICOMwebClient(url, headers=headers)

def all_studies(client):
  studies = []
  offset = 0
  while True:
    subset = client.search_for_studies(offset=offset)
    if len(subset) == 0:
        break
    studies.extend(subset)
    offset += len(subset)
  return studies

def log(message, end="\\n", flush=False):
  print(str(message), end=end, flush=flush)
  fp = open("/var/www/html/index.html", "a")
  fp.write(str(message)+end)
  fp.close()

tciaBaseURL = "https://healthcare.googleapis.com/v1beta1/projects/chc-tcia/locations/us-central1/datasets"

idcURL = "https://healthcare.googleapis.com/v1beta1/projects/idc-sandbox-000/locations/us-central1/datasets/sdp-dev/dicomStores/scratch3/dicomWeb"

collection = sys.argv[1]

bucket = "gs://dev-isomics-tcia-nii"

tciaURL = os.path.join(tciaBaseURL, collection, "dicomStores", collection, "dicomWeb")
tciaClient=freshClient(tciaURL)
studies = all_studies(tciaClient)
log(str(len(studies)) + " studies in " + collection)

for study in studies:
  tciaClient=freshClient(tciaURL)
  studyMetadata = dicomweb_client.api.load_json_dataset(study)
  log('processing %s' % studyMetadata.StudyInstanceUID)
  series = tciaClient.search_for_series(studyMetadata.StudyInstanceUID)
  log("  len(series) = %d" % len(series))
  try:
    for serie in series:
      tciaClient=freshClient(tciaURL)
      run("rm -rf /seriesInstances")
      run("rm -rf /converted")
      run("mkdir /seriesInstances")
      run("mkdir /converted")
      seriesMetadata = dicomweb_client.api.load_json_dataset(serie)
      instances = tciaClient.search_for_instances(
        study_instance_uid=studyMetadata.StudyInstanceUID,
        series_instance_uid=seriesMetadata.SeriesInstanceUID
      )
      for instance in instances:
        instanceUID = instance['00080018']
        instance = tciaClient.retrieve_instance(
          study_instance_uid=study['0020000D']['Value'][0],
          series_instance_uid=serie['0020000E']['Value'][0],
          sop_instance_uid= instance['00080018']['Value'][0]
        )
        log (instance.InstanceNumber, end=", ", flush=True)
        pydicom.write_file("/seriesInstances/%s.dcm" % instance.SOPInstanceUID, instance)
      log('done')
      cmd = "./dcm2niix "
      cmd += "-o /converted " # output dir
      cmd += "-f %s " % seriesMetadata.SeriesInstanceUID # output format
      cmd += "-c %s " % collection # comment in nii file
      cmd += "-z o " # optimize compression (with pigz)
      cmd += "/seriesInstances " # source directory
      log(cmd)
      log(run(cmd))
      files = run("find /converted -type f").split('\\n')
      log(files)
      for filePath in files:
        fileName = os.path.basename(filePath)
        cmd = "gsutil cp %s %s/%s/%s/%s" % (filePath, bucket, collection, studyMetadata.StudyInstanceUID, fileName)
        log(cmd)
        log(run(cmd))
      log('finished series')
  except:
    log('error in series')
    log(sys.exc_info())
    log(sys.exc_info()[0])
  log('finished study')
log('finished collection')

"""
# end of python template


# create a startup script template
# - this is configured by the notebook
#   and then passed as metadata to the vm
# . to use as a boot script
startupScriptTemplate = """#! /bin/bash

function log() { echo $(date) $* >> /var/www/html/index.html; }


# work
apt-get update
apt-get install -y apache2
chmod a+w /var/www/html/index.html 
echo Processing %%COLLECTION%% > /var/www/html/index.html

  log Starting
  log token $(gcloud auth print-access-token)
gsutil cp gs://dev-isomics-scripts/copyStudy-%%COLLECTION%%.py copyStudy-%%COLLECTION%%.py
  log Installing
apt-get install unzip  
wget https://github.com/rordenlab/dcm2niix/releases/download/v1.0.20190902/dcm2niix_lnx.zip
unzip dcm2niix_lnx.zip
apt-get install -y 
apt-get install -y python3-pip
pip3 install dicomweb-client
  log copying
  log output

python3 copyStudy-%%COLLECTION%%.py %%COLLECTION%%

cat /var/log/syslog >> /var/www/html/index.html

  log done, shutting down

gsutil cp /var/www/html/index.html gs://dev-isomics-tcia-nii/%%COLLECTION%%/convert-log.html


## finally have the machine delete itself
# from: https://cloud.google.com/community/tutorials/create-a-self-deleting-virtual-machine
export NAME=$(curl -X GET http://metadata.google.internal/computeMetadata/v1/instance/name -H 'Metadata-Flavor: Google')
export ZONE=$(curl -X GET http://metadata.google.internal/computeMetadata/v1/instance/zone -H 'Metadata-Flavor: Google')
gcloud --quiet compute instances delete $NAME --zone=$ZONE
"""

# Templates
# - for deleting any lingering machines
deleteTemplate = "gcloud compute instances delete instance-%%INSTANCE_NAME%% \
 --quiet --project idc-sandbox-000 --zone us-central1-a"
# - for running the actual job
# - needs cloud-platform scope to access healthcare API
# - make sure the service account is whitelisted for public healthcare datset access
# -- https://services.google.com/fb/forms/cloudhealthcarepublicdatasetaccess/
runTemplate = "gcloud compute instances create instance-%%INSTANCE_NAME%% \
 --machine-type=n1-standard-8 --async \
 --scopes=https://www.googleapis.com/auth/cloud-platform \
 --service-account=spd-dicom-service@idc-sandbox-000.iam.gserviceaccount.com \
 --tags http-server \
 --project idc-sandbox-000 \
 --zone us-central1-a \
 --metadata-from-file startup-script=startup-%%COLLECTION%%.sh"

collections = healthcareDatasets()

## optionally subset data for testing
#collections = collections[:2]

totalStudies = 0
for collection in collections:
    instanceName = collection.replace("_", "-")
    print(collection, instanceName)
    # convert this inner loop into a compute instance launch
    deleteCommand = deleteTemplate.replace("%%INSTANCE_NAME%%", instanceName)
    print(deleteCommand)
    try:
      run(deleteCommand)
    except:
      print('nothing to delete')
      pass
    pythonScript = pythonScriptTemplate.replace("%%COLLECTION%%", collection).replace("%%INSTANCE_NAME%%", instanceName).replace("%%TOKEN%%", freshToken())
    open(f'copyStudy-{collection}.py', 'w').write(pythonScript)
    run(f"gsutil cp copyStudy-{collection}.py gs://dev-isomics-scripts/copyStudy-{collection}.py")

    startupScript = startupScriptTemplate.replace("%%COLLECTION%%", collection).replace("%%INSTANCE_NAME%%", instanceName)
    open("startup-"+collection+".sh", 'w').write(startupScript)

    runCommand = runTemplate.replace("%%COLLECTION%%", collection).replace("%%INSTANCE_NAME%%", instanceName)
    print(runCommand)
    run(runCommand)

print(f"\n - Should be {len(collections)} collections in scratch3")

4d-lung 4d-lung
gcloud compute instances delete instance-4d-lung  --quiet --project idc-sandbox-000 --zone us-central1-a
nothing to delete
gcloud compute instances create instance-4d-lung  --machine-type=n1-standard-8 --async  --scopes=https://www.googleapis.com/auth/cloud-platform  --service-account=spd-dicom-service@idc-sandbox-000.iam.gserviceaccount.com  --tags http-server  --project idc-sandbox-000  --zone us-central1-a  --metadata-from-file startup-script=startup-4d-lung.sh
anti-pd-1_melanoma anti-pd-1-melanoma
gcloud compute instances delete instance-anti-pd-1-melanoma  --quiet --project idc-sandbox-000 --zone us-central1-a
nothing to delete
gcloud compute instances create instance-anti-pd-1-melanoma  --machine-type=n1-standard-8 --async  --scopes=https://www.googleapis.com/auth/cloud-platform  --service-account=spd-dicom-service@idc-sandbox-000.iam.gserviceaccount.com  --tags http-server  --project idc-sandbox-000  --zone us-central1-a  --metadata-from-file startup-script=start

In [ ]:
!gcloud compute instances list --project idc-sandbox-000

NAME                         ZONE           MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP     STATUS
instance-1                   us-central1-a  n1-standard-1               10.128.0.50  35.225.29.3     RUNNING
instance-4d-lung             us-central1-a  n1-standard-8               10.128.0.92  34.68.107.149   STOPPING
instance-anti-pd-1-melanoma  us-central1-a  n1-standard-8               10.128.0.96  35.202.107.129  RUNNING
sdp-dev-template-1           us-central1-a  n1-standard-4               10.128.0.6                   TERMINATED
slicer-build-1               us-central1-a  n1-highcpu-96               10.128.0.3                   TERMINATED


In [ ]:
!gcloud compute instances create instance-4d-lung  --machine-type=n1-standard-8 --async  --scopes=https://www.googleapis.com/auth/cloud-platform  --service-account=compute-user@project-7519307760985532298.iam.gserviceaccount.com  --tags http-server  --project project-7519307760985532298  --zone us-central1-a  --metadata-from-file startup-script=startup-4d-lung.sh


NOTE: The users will be charged for public IPs when VMs are created.
Instance creation in progress for [instance-4d-lung]: https://www.googleapis.com/compute/v1/projects/project-7519307760985532298/zones/us-central1-a/operations/operation-1578505172134-59ba4600faa5d-ecb09456-b9fb0bb4
Use [gcloud compute operations describe URI] command to check the status of the operation(s).


***Other Experiments***

# Feature Extraction

```
time  gsutil ls gs://dev-isomics-tcia-nii/\*/\* | tee series

# extract.sh
#!/bin/bash

  echo $*
  gsutil cp $1 .
  export name=$(basename $1)  
  gunzip ${name}
  export name=$(basename -s .gz ${name})  
  ./featExtract1.6/featExtract.ubu -w $name ${name}.key
  gsutil cp ${name}.key gs://dev-isomics-features
  rm ${name}
  rm ${name}.key

cat series | grep .nii.gz | xargs -n 1 -P 4 ./extract.sh

```


In [ ]:
print(run("date"))
collections = healthcareDatasets()
print(collections)

for collection in collections:
    print(run("date"))
    print(collection)
    # convert this inner loop into a compute instance launch
    print( run("gsutil -u idc-sandbox-000 du -sh gs://gcs-public-data--healthcare-tcia-%s" % collection) )

Thu Jan  9 00:35:44 UTC 2020
['4d-lung', 'anti-pd-1_melanoma', 'apollo', 'breast-diagnosis', 'breast-mri-nact-pilot', 'cbis-ddsm', 'cc-radiomics-phantom', 'cptac-ccrcc', 'cptac-cm', 'cptac-gbm', 'cptac-hnscc', 'cptac-lscc', 'cptac-luad', 'cptac-pda', 'cptac-ucec', 'ct-colonography', 'ct-lymph-nodes', 'head-neck-cetuximab', 'head-neck-pet-ct', 'hnscc', 'hnscc-3dct-rt', 'ispy1', 'ivygap', 'lctsc', 'lgg-1p19qdeletion', 'lidc-idri', 'lung-fused-ct-pathology', 'lung-phantom', 'lungct-diagnosis', 'mouse-astrocytoma', 'mouse-mammary', 'mri-dir', 'naf-prostate', 'nsclc-radiogenomics', 'nsclc-radiomics', 'nsclc-radiomics-genomics', 'pancreas-ct', 'phantom-fda', 'prostate-3t', 'prostate-diagnosis', 'prostate-fused-mri-pathology', 'prostatex', 'qiba-ct-1c', 'qin-breast-dce-mri', 'qin-headneck', 'qin-lung-ct', 'qin-pet-phantom', 'rembrandt', 'rider-breast-mri', 'rider-lung-ct', 'rider-lung-pet-ct', 'rider-neuro-mri', 'rider-phantom-mri', 'rider-phantom-pet-ct', 'soft-tissue-sarcoma', 'spie-aapm-lu

CalledProcessError: ignored

```
rock:collections pieper$ python3 collections.py 
Wed Jan  8 21:18:44 EST 2020
Wed Jan  8 21:18:44 EST 2020
4d-lung
170.47 GiB   gs://gcs-public-data--healthcare-tcia-4d-lung
Wed Jan  8 21:20:59 EST 2020
anti-pd-1_melanoma
91.65 GiB    gs://gcs-public-data--healthcare-tcia-anti-pd-1_melanoma
Wed Jan  8 21:22:30 EST 2020
apollo
2.46 GiB     gs://gcs-public-data--healthcare-tcia-apollo
Wed Jan  8 21:22:33 EST 2020
breast-diagnosis
56.69 GiB    gs://gcs-public-data--healthcare-tcia-breast-diagnosis
Wed Jan  8 21:23:14 EST 2020
breast-mri-nact-pilot
18.85 GiB    gs://gcs-public-data--healthcare-tcia-breast-mri-nact-pilot
Wed Jan  8 21:23:53 EST 2020
cbis-ddsm
152.28 GiB   gs://gcs-public-data--healthcare-tcia-cbis-ddsm
Wed Jan  8 21:23:58 EST 2020
cc-radiomics-phantom
1.33 GiB     gs://gcs-public-data--healthcare-tcia-cc-radiomics-phantom
Wed Jan  8 21:24:00 EST 2020
cptac-ccrcc
19.94 GiB    gs://gcs-public-data--healthcare-tcia-cptac-ccrcc
Wed Jan  8 21:24:15 EST 2020
cptac-cm
3.03 GiB     gs://gcs-public-data--healthcare-tcia-cptac-cm
Wed Jan  8 21:24:18 EST 2020
cptac-gbm
19.32 GiB    gs://gcs-public-data--healthcare-tcia-cptac-gbm
Wed Jan  8 21:24:42 EST 2020
cptac-hnscc
11.82 GiB    gs://gcs-public-data--healthcare-tcia-cptac-hnscc
Wed Jan  8 21:24:52 EST 2020
cptac-lscc
590.59 MiB   gs://gcs-public-data--healthcare-tcia-cptac-lscc
Wed Jan  8 21:24:54 EST 2020
cptac-luad
3.82 GiB     gs://gcs-public-data--healthcare-tcia-cptac-luad
Wed Jan  8 21:24:58 EST 2020
cptac-pda
21.78 GiB    gs://gcs-public-data--healthcare-tcia-cptac-pda
Wed Jan  8 21:25:15 EST 2020
cptac-ucec
25.53 GiB    gs://gcs-public-data--healthcare-tcia-cptac-ucec
Wed Jan  8 21:25:45 EST 2020
ct-colonography
462.56 GiB   gs://gcs-public-data--healthcare-tcia-ct-colonography
Wed Jan  8 21:30:40 EST 2020
ct-lymph-nodes
53.83 GiB    gs://gcs-public-data--healthcare-tcia-ct-lymph-nodes
Wed Jan  8 21:31:19 EST 2020
head-neck-cetuximab
48.77 GiB    gs://gcs-public-data--healthcare-tcia-head-neck-cetuximab
Wed Jan  8 21:32:37 EST 2020
head-neck-pet-ct
67.49 GiB    gs://gcs-public-data--healthcare-tcia-head-neck-pet-ct
Wed Jan  8 21:33:21 EST 2020
hnscc
104.77 GiB   gs://gcs-public-data--healthcare-tcia-hnscc
Wed Jan  8 21:36:15 EST 2020
hnscc-3dct-rt
0 B          gs://gcs-public-data--healthcare-tcia-hnscc-3dct-rt
Wed Jan  8 21:36:16 EST 2020
ispy1
72.98 GiB    gs://gcs-public-data--healthcare-tcia-ispy1
Wed Jan  8 21:38:46 EST 2020
ivygap
130.37 GiB   gs://gcs-public-data--healthcare-tcia-ivygap
Wed Jan  8 21:43:45 EST 2020
lctsc
4.76 GiB     gs://gcs-public-data--healthcare-tcia-lctsc
Wed Jan  8 21:43:49 EST 2020
lgg-1p19qdeletion
2.47 GiB     gs://gcs-public-data--healthcare-tcia-lgg-1p19qdeletion
Wed Jan  8 21:43:57 EST 2020
lidc-idri
124.83 GiB   gs://gcs-public-data--healthcare-tcia-lidc-idri
Wed Jan  8 21:45:32 EST 2020
lung-fused-ct-pathology
failed for lung-fused-ct-pathology
Wed Jan  8 21:45:33 EST 2020
lung-phantom
936.97 MiB   gs://gcs-public-data--healthcare-tcia-lung-phantom
Wed Jan  8 21:45:34 EST 2020
lungct-diagnosis
2.3 GiB      gs://gcs-public-data--healthcare-tcia-lungct-diagnosis
Wed Jan  8 21:45:36 EST 2020
mouse-astrocytoma
2.05 GiB     gs://gcs-public-data--healthcare-tcia-mouse-astrocytoma
Wed Jan  8 21:45:52 EST 2020
mouse-mammary
7.99 GiB     gs://gcs-public-data--healthcare-tcia-mouse-mammary
Wed Jan  8 21:46:02 EST 2020
mri-dir
0 B          gs://gcs-public-data--healthcare-tcia-mri-dir
Wed Jan  8 21:46:03 EST 2020
naf-prostate
12.93 GiB    gs://gcs-public-data--healthcare-tcia-naf-prostate
Wed Jan  8 21:46:27 EST 2020
nsclc-radiogenomics
90.77 GiB    gs://gcs-public-data--healthcare-tcia-nsclc-radiogenomics
Wed Jan  8 21:48:09 EST 2020
nsclc-radiomics
25.12 GiB    gs://gcs-public-data--healthcare-tcia-nsclc-radiomics
Wed Jan  8 21:48:31 EST 2020
nsclc-radiomics-genomics
6.61 GiB     gs://gcs-public-data--healthcare-tcia-nsclc-radiomics-genomics
Wed Jan  8 21:48:37 EST 2020
pancreas-ct
9.46 GiB     gs://gcs-public-data--healthcare-tcia-pancreas-ct
Wed Jan  8 21:48:46 EST 2020
phantom-fda
728.5 GiB    gs://gcs-public-data--healthcare-tcia-phantom-fda
Wed Jan  8 21:57:50 EST 2020
prostate-3t
283.92 MiB   gs://gcs-public-data--healthcare-tcia-prostate-3t
Wed Jan  8 21:57:53 EST 2020
prostate-diagnosis
5.57 GiB     gs://gcs-public-data--healthcare-tcia-prostate-diagnosis
Wed Jan  8 21:58:07 EST 2020
prostate-fused-mri-pathology
4.42 GiB     gs://gcs-public-data--healthcare-tcia-prostate-fused-mri-pathology
Wed Jan  8 21:58:21 EST 2020
prostatex
15.07 GiB    gs://gcs-public-data--healthcare-tcia-prostatex
Wed Jan  8 22:00:12 EST 2020
qiba-ct-1c
34 GiB       gs://gcs-public-data--healthcare-tcia-qiba-ct-1c
Wed Jan  8 22:00:39 EST 2020
qin-breast-dce-mri
14.81 GiB    gs://gcs-public-data--healthcare-tcia-qin-breast-dce-mri
Wed Jan  8 22:01:10 EST 2020
qin-headneck
95.95 GiB    gs://gcs-public-data--healthcare-tcia-qin-headneck
Wed Jan  8 22:03:29 EST 2020
qin-lung-ct
2.27 GiB     gs://gcs-public-data--healthcare-tcia-qin-lung-ct
Wed Jan  8 22:03:34 EST 2020
qin-pet-phantom
234.89 MiB   gs://gcs-public-data--healthcare-tcia-qin-pet-phantom
Wed Jan  8 22:03:36 EST 2020
rembrandt
9.87 GiB     gs://gcs-public-data--healthcare-tcia-rembrandt
Wed Jan  8 22:04:20 EST 2020
rider-breast-mri
383 MiB      gs://gcs-public-data--healthcare-tcia-rider-breast-mri
Wed Jan  8 22:04:22 EST 2020
rider-lung-ct
8.24 GiB     gs://gcs-public-data--healthcare-tcia-rider-lung-ct
Wed Jan  8 22:04:29 EST 2020
rider-lung-pet-ct
77.55 GiB    gs://gcs-public-data--healthcare-tcia-rider-lung-pet-ct
Wed Jan  8 22:06:08 EST 2020
rider-neuro-mri
7.25 GiB     gs://gcs-public-data--healthcare-tcia-rider-neuro-mri
Wed Jan  8 22:06:35 EST 2020
rider-phantom-mri
3.36 GiB     gs://gcs-public-data--healthcare-tcia-rider-phantom-mri
Wed Jan  8 22:06:38 EST 2020
rider-phantom-pet-ct
0 B          gs://gcs-public-data--healthcare-tcia-rider-phantom-pet-ct
Wed Jan  8 22:06:39 EST 2020
soft-tissue-sarcoma
9.19 GiB     gs://gcs-public-data--healthcare-tcia-soft-tissue-sarcoma
Wed Jan  8 22:06:55 EST 2020
spie-aapm-lung-ct-challenge
11.24 GiB    gs://gcs-public-data--healthcare-tcia-spie-aapm-lung-ct-challenge
Wed Jan  8 22:07:05 EST 2020
tcga-blca
37.34 GiB    gs://gcs-public-data--healthcare-tcia-tcga-blca
Wed Jan  8 22:07:36 EST 2020
tcga-brca
82.08 GiB    gs://gcs-public-data--healthcare-tcia-tcga-brca
Wed Jan  8 22:09:08 EST 2020
tcga-cesc
8.85 GiB     gs://gcs-public-data--healthcare-tcia-tcga-cesc
Wed Jan  8 22:09:16 EST 2020
tcga-coad
4.19 GiB     gs://gcs-public-data--healthcare-tcia-tcga-coad
Wed Jan  8 22:09:20 EST 2020
tcga-esca
10.19 GiB    gs://gcs-public-data--healthcare-tcia-tcga-esca
Wed Jan  8 22:09:29 EST 2020
tcga-gbm
68.47 GiB    gs://gcs-public-data--healthcare-tcia-tcga-gbm
Wed Jan  8 22:12:23 EST 2020
tcga-hnsc
120.72 GiB   gs://gcs-public-data--healthcare-tcia-tcga-hnsc
Wed Jan  8 22:14:09 EST 2020
tcga-kich
4.25 GiB     gs://gcs-public-data--healthcare-tcia-tcga-kich
Wed Jan  8 22:14:14 EST 2020
tcga-kirc
85.28 GiB    gs://gcs-public-data--healthcare-tcia-tcga-kirc
Wed Jan  8 22:15:29 EST 2020
tcga-kirp
9.56 GiB     gs://gcs-public-data--healthcare-tcia-tcga-kirp
Wed Jan  8 22:15:40 EST 2020
tcga-lgg
39.97 GiB    gs://gcs-public-data--healthcare-tcia-tcga-lgg
Wed Jan  8 22:17:15 EST 2020
tcga-lihc
52.5 GiB     gs://gcs-public-data--healthcare-tcia-tcga-lihc
Wed Jan  8 22:18:07 EST 2020
tcga-luad
18.27 GiB    gs://gcs-public-data--healthcare-tcia-tcga-luad
Wed Jan  8 22:18:26 EST 2020
tcga-lusc
13.99 GiB    gs://gcs-public-data--healthcare-tcia-tcga-lusc
Wed Jan  8 22:18:41 EST 2020
tcga-ov
26.36 GiB    gs://gcs-public-data--healthcare-tcia-tcga-ov
Wed Jan  8 22:19:01 EST 2020
tcga-prad
3.48 GiB     gs://gcs-public-data--healthcare-tcia-tcga-prad
Wed Jan  8 22:19:09 EST 2020
tcga-read
403.87 MiB   gs://gcs-public-data--healthcare-tcia-tcga-read
Wed Jan  8 22:19:11 EST 2020
tcga-sarc
failed for tcga-sarc
Wed Jan  8 22:19:11 EST 2020
tcga-stad
21.72 GiB    gs://gcs-public-data--healthcare-tcia-tcga-stad
Wed Jan  8 22:19:28 EST 2020
tcga-thca
1.08 GiB     gs://gcs-public-data--healthcare-tcia-tcga-thca
Wed Jan  8 22:19:30 EST 2020
tcga-ucec
35.25 GiB    gs://gcs-public-data--healthcare-tcia-tcga-ucec


```



In [ ]:
%%bash


export PROJECT_ID=chc-tcia
export REGION=us-central1
export DATASET_ID=ct-colonography
export DICOM_STORE_ID=ct-colonography

curl -X GET --silent \
     -H "Authorization: Bearer "$(gcloud auth print-access-token) \
     "https://healthcare.googleapis.com/v1beta1/projects/$PROJECT_ID/locations/$REGION/datasets/$DATASET_ID/dicomStores/$DICOM_STORE_ID/dicomWeb/studies" \
     -o /tmp/studies
     
wc /tmp/studies


    1    81 50452 /tmp/studies


In [ ]:

tciaBaseURL = f"https://healthcare.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{REGION}/datasets"

idcURL = "https://healthcare.googleapis.com/v1beta1/projects/idc-sandbox-000/locations/us-central1/datasets/sdp-dev/dicomStores/scratch3/dicomWeb"

totalStudies = 0
for collection in collections:
    tciaURL = f"{tciaBaseURL}/{collection}/dicomStores/{collection}/dicomWeb"
    tciaClient=freshClient(tciaURL)
    studies = all_studies(tciaClient)
    print(f"{len(studies)} studies in {collection}")
    totalStudies += len(studies)
    if len(studies) > 0:
      study = random.sample(studies,1)[0]
      print(study)
      studyUID = study['0020000D']['Value'][0]
      print(f"retrieving {studyUID}...")
      instances = tciaClient.retrieve_study(studyUID)
      idcClient = freshClient(idcURL)
      print(f"storing...")
      idcClient.store_instances(instances)
      print(f"There are {totalStudies} total")

comment = """
for study in studies:
  studyMetadata = dicomweb_client.api.load_json_dataset(study)
  if hasattr(studyMetadata, "ModalitiesInStudy"):
    print(f"Modalities: {studyMetadata.ModalitiesInStudy}")
  else:
    print("Modalities not defined")
  series = client.search_for_series(studyMetadata.StudyInstanceUID)
  print(f"  {len(series)} series")
  for serie in series:
    seriesMetadata = dicomweb_client.api.load_json_dataset(serie)
    instances = client.search_for_instances(
      study_instance_uid=studyMetadata.StudyInstanceUID,
      series_instance_uid=seriesMetadata.SeriesInstanceUID
    )
    print(f"    {len(instances)} instances ", end="", flush=True)
  print()
"""


(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI']
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.14519.5.2.1.2744.7002.100859319058233426582393711948
(0008, 0020) Study Date                          DA: '19851210'
(0008, 0021) Series Date                         DA: '19851210'
(0008, 0022) Acquisition Date                    DA: '19851210'
(0008, 0023) Content Date                        DA: '19851210'
(0008, 0030) Study Time                          TM: '131911.063000'
(0008, 0031) Series Time                         TM: '132125.647000'
(0008, 0032) Acquisition Time                    TM: '132241.255005'
(0008, 0033) Content Time                        TM: '132241.255005'
(0008, 0050) Accession Number                    SH: '2076699673350889'
(0008, 0060) Modality                    

In [ ]:
# Move a single study from one dicomstore to another

tciaURL = f"https://healthcare.googleapis.com/v1beta1/projects/chc-tcia/locations/us-central1/datasets/ct-lymph-nodes/dicomStores/ct-lymph-nodes/dicomWeb"
studyUID = f"61.7.22285965616260355338860879829667630274"
idcURL = "https://healthcare.googleapis.com/v1beta1/projects/idc-sandbox-000/locations/us-central1/datasets/sdp-dev/dicomStores/scratch/dicomWeb"

tciaClient=freshClient(tciaURL)
print(f"fetching...")
instances = tciaClient.retrieve_study(studyUID)
idcClient = freshClient(idcURL)
print(f"storing...")
idcClient.store_instances(instances)
print(f"done.")


fetching...
storing...
done.


In [ ]:
%%bash

# sudo apt-get install parallel

#time  gsutil ls gs://dev-isomics-tcia-nii/\*/\* | tee series
# 

cat << EOF > extract.sh
#!/bin/bash
  echo $*
  gsutil cp $1 .
  export name=$(basename $1)  
  gunzip ${name}
  export name=$(basename -s .gz ${name})  
  ./featExtract1.6/featExtract.ubu -w $name ${name}.key
  gsutil cp ${name}.key gs://dev-isomics-features
  rm ${name}
  rm ${name}.key

EOF

./extract.sh gs://dev-isomics-tcia-nii/4d-lung/1.3.6.1.4.1.14519.5.2.1.6834.5010.101259051157428667137154755870/1.3.6.1.4.1.14519.5.2.1.6834.5010.110117618917781740286980097865.nii.gz


time cat series | grep ct-lymph | head -100 | grep .nii.gz | parallel -j 4 ./extract.sh

time cat series | grep .nii.gz | xargs -n 1 -P 4  ./extract.sh 2>&1 | tee runlog.txt 


basename: missing operand
Try 'basename --help' for more information.
basename: missing operand
Try 'basename --help' for more information.
bash: line 18: ./extract.sh: Permission denied
bash: line 20: parallel: command not found
head: cannot open 'series' for reading: No such file or directory
